# Reconstructing historical portfolio information from Coinbase Pro

Recently I've started doing some very small scale cryptocurrency trading using [Coinbase Pro](https://pro.coinbase.com/). Currently my method of keeping track of my historical orders is by duplicating the information found on the Orders tab. One thing that Coinbase Pro doesn't do however is keep track of your historical portfolio *value* (like Coinbase non-Pro does) or the historical constituents of the portfolio. In this notebook I'll reconstruct these values from a list of trades and deposits mad